In [ ]:
import duckdb
import pandas as pd

from bs4 import BeautifulSoup


# dev_mode = True
dev_mode = False
if dev_mode:
    # DEV (user specific)
    database = "/home/heiler/development/projects/ascii/research-space/src/pipelines/ascii/ascii_dbt/ascii_pipeline.duckdb"
    prefix = "ascii_dev"
else:
    # prod
    database = "/data/raid5/data/ascii/mastered-data/ascii_pipeline.duckdb"
    prefix = "ascii"

con = duckdb.connect(
    database=database,
    read_only=True,
)

In [ ]:
%store -r df_urls_filtered

In [ ]:
df = df_urls_filtered

In [ ]:
df["ascii_id_company"].nunique()

In [ ]:
len(df)

## Retrieve content from remaining links

In [ ]:
# # Create a temporary table for the URLs and ascii_id_company
# create_temp_table_query = "CREATE TEMPORARY TABLE temp_urls (src_url VARCHAR, ascii_id_company VARCHAR)"
# con.execute(create_temp_table_query)

# # Insert the URLs and ascii_id_company from processed_df into the temporary table
# for _, row in df.iterrows():
#     insert_query = f"INSERT INTO temp_urls VALUES ('{row['src_url']}', '{row['ascii_id_company']}')"
#     con.execute(insert_query)

In [ ]:
# Create a temporary table for the URLs and ascii_id_company
create_temp_table_query = (
    "CREATE TEMPORARY TABLE temp_urls (src_url VARCHAR, ascii_id_company VARCHAR)"
)
con.execute(create_temp_table_query)

# Insert the URLs and ascii_id_company from processed_df into the temporary table
for _, row in df.iterrows():
    insert_query = "INSERT INTO temp_urls (src_url, ascii_id_company) VALUES (?, ?)"
    con.execute(insert_query, (row["src_url"], row["ascii_id_company"]))

In [ ]:
con.query("select * from temp_urls limit 10")

In [ ]:
# Query to join the temporary table with edges and get content along with ascii_id_company
content_query = """
SELECT t.ascii_id_company, t.src_url, e.content AS content
FROM temp_urls t
JOIN ascii_commoncrawl.edges e ON t.src_url = e.src_url
WHERE e.seed_nodes = 'Georgetown_v1.0.0' AND e.crawl_id = 'CC-MAIN-2021-49'
"""


# Execute the query and fetch the result into a DataFrame WARNING takes long time! around 1m
content_df = con.execute(content_query).fetchdf()

In [ ]:
content_df

In [ ]:
print(len(content_df))
print(len(df))

ok we got some duplicated urls, those we will drop and just keep the first

In [ ]:
content_df = content_df.drop_duplicates(subset="src_url", keep="first")

In [ ]:
print(len(content_df))
print(len(df))

In [ ]:
%store content_df